In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("basic") \
    .master("local[*]") \
    .getOrCreate()

In [3]:
spark

In [4]:
spark.conf.get("spark.sql.shuffle.partitions")

'200'

In [5]:
spark.conf.get("spark.sql.adaptive.enabled")

'true'

In [31]:
import psycopg2
import os

conn = psycopg2.connect(
    host="localhost",
    port=5432,
    database="interview_prep",
    user="postgres",
    password="@impandit"
)

In [38]:
cur = conn.cursor()

In [39]:
cur.execute("SELECT version();")

In [40]:
print(cur.fetchone())

('PostgreSQL 16.9, compiled by Visual C++ build 1943, 64-bit',)


In [41]:
cur.execute("set search_path to swiss_re;")

In [44]:
print(cur.execute("select * from swiss_re.claims;"))

None


In [36]:
print(data)

None


In [37]:
cur.close()

In [30]:
conn.close()

In [1]:
from faker import Faker
import pandas as pd
import random
from datetime import datetime

fake = Faker()

# Constants
num_records = 10000
source_system = "Contract_SR_Europa_3"
contract_types = ["Direct", "Reinsurance", ""]  # include blank as valid

# Fixed Dates
insured_from = "01.01.2015"
insured_to = "01.01.2099"
creation_date = "17.01.2022 13:42"

# Generate Data
data = []

for _ in range(num_records):
    contract_id = random.randint(10000000, 999999999)  # 8 to 9 digit number
    contract_type = random.choices(contract_types, weights=[0.45, 0.45, 0.1])[0]  # blank 10% of the time

    row = {
        "SOURCE_SYSTEM": source_system,
        "CONTRACT_ID": contract_id,
        "CONTRACT_TYPE": contract_type,
        "INSURED_PERIOD_FROM": insured_from,
        "INSUDRED_PERIOD_TO": insured_to,
        "CREATION_DATE": creation_date
    }
    data.append(row)

# Create DataFrame
df = pd.DataFrame(data)
# 
# # Save to CSV
# df.to_csv("contract_data_10k.csv", index=False)


In [2]:
df.head()

,SOURCE_SYSTEM,CONTRACT_ID,CONTRACT_TYPE,INSURED_PERIOD_FROM,INSUDRED_PERIOD_TO,CREATION_DATE
0,Contract_SR_Europa_3,537904473,Direct,01.01.2015,01.01.2099,17.01.2022 13:42
1,Contract_SR_Europa_3,509721623,Direct,01.01.2015,01.01.2099,17.01.2022 13:42
2,Contract_SR_Europa_3,756773014,Direct,01.01.2015,01.01.2099,17.01.2022 13:42
3,Contract_SR_Europa_3,929285796,Reinsurance,01.01.2015,01.01.2099,17.01.2022 13:42
4,Contract_SR_Europa_3,695527691,Direct,01.01.2015,01.01.2099,17.01.2022 13:42


In [3]:
import pandas as pd
import random
from faker import Faker
from datetime import datetime

fake = Faker()

# Load contract_id list from earlier file
contracts_df = df.copy()
valid_contract_ids = contracts_df["CONTRACT_ID"].tolist()

num_records = 10000
match_percent = 0.9
num_matching = int(num_records * match_percent)
num_non_matching = num_records - num_matching

claim_ids = set()
data = []

# Function to create unique claim ID
def generate_claim_id():
    while True:
        prefix = random.choice(["CL_", "CX_", "RX_", "U_"])
        num = random.randint(100000, 99999999)
        claim_id = prefix + str(num)
        if claim_id not in claim_ids:
            claim_ids.add(claim_id)
            return claim_id

# Generate matching CONTRACT_IDs (90%)
for _ in range(num_matching):
    row = {
        "SOURCE_SYSTEM": "Claim_SR_Europa_3",
        "CLAIM_ID": generate_claim_id(),
        "CONTRACT_SOURCE_SYSTEM": "Contract_SR_Europa_3",
        "CONTRACT_ID": random.choice(valid_contract_ids),
        "CLAIM_TYPE": random.choices([1, 2, ""], weights=[0.45, 0.45, 0.1])[0],
        "DATE_OF_LOSS": fake.date_between(start_date='-5y', end_date='today').strftime("%d.%m.%Y"),
        "AMOUNT": round(random.uniform(100.0, 100000.0), 2),
        "CREATION_DATE": "17.01.2022 14:45"
    }
    data.append(row)

# Generate non-matching CONTRACT_IDs (10%)
for _ in range(num_non_matching):
    row = {
        "SOURCE_SYSTEM": "Claim_SR_Europa_3",
        "CLAIM_ID": generate_claim_id(),
        "CONTRACT_SOURCE_SYSTEM": "Contract_SR_Europa_3",
        "CONTRACT_ID": random.randint(10000000, 999999999),  # unlikely to match
        "CLAIM_TYPE": random.choices([1, 2, ""], weights=[0.45, 0.45, 0.1])[0],
        "DATE_OF_LOSS": fake.date_between(start_date='-5y', end_date='today').strftime("%d.%m.%Y"),
        "AMOUNT": round(random.uniform(100.0, 100000.0), 2),
        "CREATION_DATE": "17.01.2022 14:45"
    }
    data.append(row)

# Create DataFrame and save
claims_df = pd.DataFrame(data)
# claims_df.to_csv("claim_data_10k.csv", index=False)


In [4]:
claims_df.head()

,SOURCE_SYSTEM,CLAIM_ID,CONTRACT_SOURCE_SYSTEM,CONTRACT_ID,CLAIM_TYPE,DATE_OF_LOSS,AMOUNT,CREATION_DATE
0,Claim_SR_Europa_3,CX_95589333,Contract_SR_Europa_3,973836969,1,29.07.2020,576.42,17.01.2022 14:45
1,Claim_SR_Europa_3,CL_18933415,Contract_SR_Europa_3,771630360,2,27.11.2022,1532.53,17.01.2022 14:45
2,Claim_SR_Europa_3,CX_32667818,Contract_SR_Europa_3,890602621,2,11.02.2021,99694.64,17.01.2022 14:45
3,Claim_SR_Europa_3,U_82340382,Contract_SR_Europa_3,378874195,,08.03.2023,93608.35,17.01.2022 14:45
4,Claim_SR_Europa_3,RX_7795553,Contract_SR_Europa_3,343567665,2,27.09.2020,3434.98,17.01.2022 14:45


In [5]:
claims_df.dtypes

SOURCE_SYSTEM              object
CLAIM_ID                   object
CONTRACT_SOURCE_SYSTEM     object
CONTRACT_ID                 int64
CLAIM_TYPE                 object
DATE_OF_LOSS               object
AMOUNT                    float64
CREATION_DATE              object
dtype: object